In [1]:
library(gplots)
library(ggplot2)
library(ape)
library(phylolm);


Attaching package: 'gplots'


The following object is masked from 'package:stats':

    lowess




In [24]:
#Treat SL as 0 and SM as 1
species = c('Trichechus_manatus','Equus_caballus','Canis_lupus','Spermophilus_dauricus','Ursus_maritimus','Pteropus_vampyrus','Elephantulus_edwardii','Sus_scrofa','Odocoileus_virginianus','Oryctolagus_cuniculus','Ovis_aries','Lemur_catta','Tupaia_chinensis','Enhydra_lutris','Felis_catus','Cavia_porcellus')
coneType = c(0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,1)
traitF <- data.frame(species = species, coneType = coneType)
rownames(traitF) <- traitF$species
traitM <- as.matrix(traitF)
traitM

,species,coneType
Trichechus_manatus,Trichechus_manatus,0
Equus_caballus,Equus_caballus,0
Canis_lupus,Canis_lupus,1
Spermophilus_dauricus,Spermophilus_dauricus,1
Ursus_maritimus,Ursus_maritimus,0
Pteropus_vampyrus,Pteropus_vampyrus,1
Elephantulus_edwardii,Elephantulus_edwardii,0
Sus_scrofa,Sus_scrofa,0
Odocoileus_virginianus,Odocoileus_virginianus,1
Oryctolagus_cuniculus,Oryctolagus_cuniculus,1


In [3]:
zoonomiaPath = '/Users/tinaryu/brain_epigenetics/project/data/Zoonomia_ChrX_lessGC40_241species_30Consensus.tree'
zoonomiaTree <- read.tree(file = zoonomiaPath)
removeSpecV <- setdiff(zoonomiaTree$tip.label,species)

zoonomiaTreeDrop <- drop.tip(zoonomiaTree,removeSpecV)
zoonomiaTreeDrop$tip.label

[1] "Trichechus_manatus"     "Elephantulus_edwardii"  "Oryctolagus_cuniculus" 
 [4] "Cavia_porcellus"        "Spermophilus_dauricus"  "Tupaia_chinensis"      
 [7] "Lemur_catta"            "Pteropus_vampyrus"      "Sus_scrofa"            
[10] "Odocoileus_virginianus" "Ovis_aries"             "Equus_caballus"        
[13] "Felis_catus"            "Canis_lupus"            "Ursus_maritimus"       
[16] "Enhydra_lutris"

In [4]:
pathPeak = '/Users/tinaryu/brain_epigenetics/project/data/L23ITpeakPredM.csv'
pathSpecies = '/Users/tinaryu/brain_epigenetics/project/data/L23ITspeciesNames.csv'

In [5]:
speciesInfoF <- read.csv(file=pathSpecies,stringsAsFactors=F,header=T);
peakInfoF <- read.csv(file=pathPeak,header=TRUE,stringsAsFactors=F);


In [6]:
rownames(peakInfoF) <- peakInfoF$X
peakInfoSubsetF = peakInfoF[,species,drop=FALSE]
peakInfoSubsetM = as.matrix(peakInfoSubsetF)
peakInfoSubsetM[1:5,]

,Trichechus_manatus,Equus_caballus,Canis_lupus,Spermophilus_dauricus,Ursus_maritimus,Pteropus_vampyrus,Elephantulus_edwardii,Sus_scrofa,Odocoileus_virginianus,Oryctolagus_cuniculus,Ovis_aries,Lemur_catta,Tupaia_chinensis,Enhydra_lutris,Felis_catus,Cavia_porcellus
hg38:chr1:827294-827795,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.896454,NA,NA,NA,NA
hg38:chr1:1019232-1019733,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
hg38:chr1:2414331-2414832,NA,0.1736450,0.274206,0.690077,0.389811,0.970039,NA,NA,0.874791,NA,0.276168,NA,0.946244,0.858548,NA,0.924502
hg38:chr1:32886478-32886979,0.946243,0.9631820,0.286944,0.439608,0.489087,0.971063,0.907109,0.940642,0.930320,0.942203,0.161447,0.702246,0.350648,0.798215,0.219556,NA
hg38:chr3:51340395-51340896,0.933330,0.0719499,NA,0.849795,NA,0.103207,NA,0.956646,0.936982,0.975540,0.856439,0.922376,0.945567,NA,0.871231,NA


In [26]:
########## Loop through the peaks and calculate dichromacy relationship with phylolm ###############
#This could take a few minutes

testPeaksV <- rownames(peakInfoSubsetF);

loopPeaksV <- testPeaksV[1:1000];

peakPhyloResultsF <- data.frame(peakId = loopPeaksV, pvalue=rep(NA,length(loopPeaksV)),correlation=rep(NA,length(loopPeaksV)),adjCorrelation=rep(NA,length(loopPeaksV)));

rownames(peakPhyloResultsF) <- peakPhyloResultsF$peakId


options(warn=-1)

for(curPeak in loopPeaksV) {
  traitFtmp <- traitF;
  traitFtmp$curPeak <- peakInfoSubsetM[curPeak,];
  print(traitFtmp)
  print(zoonomiaTreeDrop$tip.label)
  curLmFit = phylolm(curPeak~coneType,data=traitFtmp,phy=zoonomiaTreeDrop,method="BM");
  curLmFitSum <- summary(curLmFit)
  peakPhyloResultsF[curPeak,"pvalue"] <- curLmFitSum$coefficients["longevity","p.value"]
  peakPhyloResultsF[curPeak,"correlation"] <- curLmFitSum$r.squared
  peakPhyloResultsF[curPeak,"adjCorrelation"] <- curLmFitSum$adj.r.squared
} 

options(warn=0)

peakPhyloResults2F <- peakPhyloResultsF[order(peakPhyloResultsF$pvalue,decreasing=F),]
peakPhyloResults2F[1:10,]


                                      species coneType  curPeak
Trichechus_manatus         Trichechus_manatus        0       NA
Equus_caballus                 Equus_caballus        0       NA
Canis_lupus                       Canis_lupus        1       NA
Spermophilus_dauricus   Spermophilus_dauricus        1       NA
Ursus_maritimus               Ursus_maritimus        0       NA
Pteropus_vampyrus           Pteropus_vampyrus        1       NA
Elephantulus_edwardii   Elephantulus_edwardii        0       NA
Sus_scrofa                         Sus_scrofa        0       NA
Odocoileus_virginianus Odocoileus_virginianus        1       NA
Oryctolagus_cuniculus   Oryctolagus_cuniculus        1       NA
Ovis_aries                         Ovis_aries        0       NA
Lemur_catta                       Lemur_catta        0 0.896454
Tupaia_chinensis             Tupaia_chinensis        0       NA
Enhydra_lutris                 Enhydra_lutris        0       NA
Felis_catus                       Felis_

ERROR: Error in phylolm(curPeak ~ coneType, data = traitFtmp, phy = zoonomiaTreeDrop, : names of the data do not match with tip labels.


In [93]:
unmatched <- setdiff(rownames(traitF), zoonomiaTreeDrop$tip.label)
if(length(unmatched) > 0) {
    print("Unmatched species:")
    print(unmatched)
} else {
    print("All species match!")
}

[1] "All species match!"


[1] "Trichechus_manatus"     "Equus_caballus"         "Canis_lupus"           
 [4] "Spermophilus_dauricus"  "Ursus_maritimus"        "Pteropus_vampyrus"     
 [7] "Elephantulus_edwardii"  "Sus_scrofa"             "Odocoileus_virginianus"
[10] "Oryctolagus_cuniculus"  "Ovis_aries"             "Lemur_catta"           
[13] "Tupaia_chinensis"       "Enhydra_lutris"         "Felis_catus"           
[16] "Cavia_porcellus"

In [59]:
tip_labels <- zoonomiaTreeDrop$tip.label
print(tip_labels)

 [1] "Trichechus_manatus"     "Elephantulus_edwardii"  "Oryctolagus_cuniculus" 
 [4] "Cavia_porcellus"        "Spermophilus_dauricus"  "Tupaia_chinensis"      
 [7] "Lemur_catta"            "Pteropus_vampyrus"      "Sus_scrofa"            
[10] "Odocoileus_virginianus" "Ovis_aries"             "Equus_caballus"        
[13] "Felis_catus"            "Canis_lupus"            "Ursus_maritimus"       
[16] "Enhydra_lutris"        


In [63]:
unique_species <- traitF$species  # Replace 'species_column_name' with the actual name of your species column
unique_species

[1] "Trichechus_manatus"     "Equus_caballus"         "Canis_lupus"           
 [4] "Spermophilus_dauricus"  "Ursus_maritimus"        "Pteropus_vampyrus"     
 [7] "Elephantulus_edwardii"  "Sus_scrofa"             "Odocoileus_virginianus"
[10] "Oryctolagus_cuniculus"  "Ovis_aries"             "Lemur_catta"           
[13] "Tupaia_chinensis"       "Enhydra_lutris"         "Felis_catus"           
[16] "Cavia_porcellus"

In [66]:
length(tip_labels)

[1] 16

In [67]:
traitFtmp <- traitF[traitF$species %in% tip_labels, ]



In [78]:
options(warn=-1)

for (curPeak in loopPeaksV) {
  traitFtmp <- traitF
  traitFtmp$curPeak <- peakInfoSubsetM[curPeak, ]
  
  # Filter to ensure only species in the phylogenetic tree are included
  traitFtmp <- traitFtmp[traitFtmp$species %in% tip_labels, ]
  

  curLmFit = phyloglm(curPeak ~ coneType, data = traitFtmp, phy = zoonomiaTreeDrop, method = "BM")
  curLmFitSum <- summary(curLmFit)
  
  peakPhyloResultsF[curPeak, "pvalue"] <- curLmFitSum$coefficients["coneType", "Pr(>|t|)"]
  peakPhyloResultsF[curPeak, "correlation"] <- curLmFitSum$r.squared
  peakPhyloResultsF[curPeak, "adjCorrelation"] <- curLmFitSum$adj.r.squared
}

options(warn=0)

peakPhyloResults2F <- peakPhyloResultsF[order(peakPhyloResultsF$pvalue, decreasing = FALSE), ]
peakPhyloResults2F[1:10, ]

ERROR: Error in phylolm(curPeak ~ coneType, data = traitFtmp, phy = zoonomiaTreeDrop, : names of the data do not match with tip labels.
